#### Análise de Séries Temporais

#### Conteúdo - Bases e Notebook da aula

#### Importação de pacotes, bibliotecas e funções (def)

In [ ]:
# Importar biblioteca completa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 

# Importar função especifica de um módulo
from statsmodels.tsa.seasonal import seasonal_decompose

#### Aula 1 - O Problema de Time Series

#### Aula 2 - Utilizando Datas como Índice

#### Aula 3 - Bibliotecas Fundamentais em Time Series Problem

#### Aula 4 - Análise Exploratória de Dados (EDA)

#### Aula 5 - Modelando e Realizando o Forecasting